In [2]:
# Module: RCSA Scheduler
# Author: Shantanu Jain
# Date: Aug 15, 2023
# Description: This module provides functions to generate schedules for members based on their 
# availabilitiy and prefrences.


import pandas as pd
import glob
import re

def insert_name_by_preference(names_list, name, preference):
    """Insert a name into a list based on preference order."""
    preference_order = {
        'B1': 0, 'A1': 1, 'A2': 2, 'B2': 3, 'B': 4,
        'A': 5, 'D1': 6, 'D2': 7, 'N': 8
    }
    
    insert_index = next((index for index, (_, pref) in enumerate(names_list)
                         if preference_order[pref] > preference_order[preference]), len(names_list))
    
    names_list.insert(insert_index, (name, preference))
    
    return names_list

def find_total_available_hours(df):
    """Find the total number of hours a person is available each week."""
    name_counts = {}
    for cell_list in df.values.flatten():
        for name, _ in cell_list:
            name_counts[name] = name_counts.get(name, 0) + 1
    return name_counts

# Initialize data and DataFrame
data = [[[] for _ in range(5)] for _ in range(9)]
columns = ["10-11AM", "11-12PM", "12-1PM", "1-2PM", "2-3PM", "3-4PM", "4-5PM", "5-6PM", "6-7PM"]
labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
availability_df = pd.DataFrame(data, columns=labels, index=columns)
people_dict = {}

# Process files and populate DataFrame
for filename in glob.glob("*.xlsx"):
    df = pd.read_excel(filename, engine='openpyxl')
    person_name = df.columns[0]
    people_dict[person_name] = 3  # Sets 3 hours to each person
    
    for col in range(5):
        for row in range(9):
            cell_val = str(df.iloc[row, col + 1])
            
            if "B" in cell_val or "A" in cell_val:
                cell_val = cell_val.replace('Y', '')  # removes Y from any string.
                cell_val = re.sub(r'(?<=\d)\d$', '', cell_val)  # removes last digit if two digits are present.
                availability_df.iloc[row, col] = insert_name_by_preference(availability_df.iloc[row, col], person_name, cell_val)

# Display results
print(find_total_available_hours(availability_df))
#print(availability_df)
print(people_dict)


{}
{}


In [5]:
#Version 8/27/2023 (Latest)

import heapq
import numpy as np

# Assuming availibility_df and people_dict are already defined in code
finalized_df = pd.DataFrame(columns=labels, index=columns)
availability_hours = find_total_available_hours(availability_df)

# Convert availability_hours into a heap structure
availability_heap = [(-hours, person) for person, hours in availability_hours.items()]
heapq.heapify(availability_heap)


# Function to check if a cell in finalized_df is empty
def is_cell_empty(row, col, df):
    """Check if a cell in the dataframe is empty."""
    return pd.isna(df.iloc[row, col])

# Function to find person with least total hours from a given list of tuples
def find_least_hours(persons):
    """Find the person with the least total available hours from a list."""
    least_hours = float('inf')
    selected_person = None

    for person in persons:
        name = person[0]
        hours = availability_hours[name]
        if hours < least_hours:
            selected_person = name
            least_hours = hours

    return selected_person

def all_slots_filled(df):
    """Check if all slots in the dataframe have been filled."""
    return not df.isna().any().any()

def update_person_availability(person_to_remove):
    """Update the availability dataframe by removing a person."""
    for row in range(availibility_df.shape[0]):
        for col in range(availibility_df.shape[1]):
            availability_df.iloc[row, col] = [p for p in availability_df.iloc[row, col] if p != person_to_remove]


# Modify the logic for removing the assigned person from the entire availability matrix
def remove_person_from_availability(person_to_remove, availability_df):
    """Remove a person from the availability dataframe if they've been fully assigned."""
    if people_dict[person_to_remove] == 0:
        for row in range(availability_df.shape[0]):
            for col in range(availability_df.shape[1]):
                availability_df.iloc[row, col] = [p for p in availability_df.iloc[row, col] if p[0] != person_to_remove]

def fill_cells_by_count(count, finalized_df):
    """Fill the finalized dataframe cells based on people count availability."""
    changed_in_this_run = False
    to_fill = finalized_df[availability_df.applymap(len) == count].isna()
    for row, col in zip(*np.where(to_fill)):
        available_people = availability_df.at[columns[row], labels[col]]
        if len(available_people) == count:
            while availability_heap:
                _, person_to_assign = heapq.heappop(availability_heap)
                # Ensure person is available and has remaining assignments
                if person_to_assign in [p[0] for p in available_people] and people_dict[person_to_assign] > 0:
                    finalized_df.at[columns[row], labels[col]] = person_to_assign
                    people_dict[person_to_assign] -= 1
                    remove_person_from_availability(person_to_assign, availability_df)
                    # Add back to heap if there are still slots left for this person
                    if people_dict[person_to_assign] > 0:
                        heapq.heappush(availability_heap, (-availability_hours[person_to_assign], person_to_assign))
                    changed_in_this_run = True
                    break
    return changed_in_this_run


def fill_remaining_slots(finalized_df):
    """Fill any remaining unfilled slots in the finalized dataframe."""
    for row in range(finalized_df.shape[0]):
        for col in range(finalized_df.shape[1]):
            if is_cell_empty(row, col, finalized_df) and availability_df.iloc[row, col]:
                person_to_assign = find_least_hours(availability_df.iloc[row, col])
                if people_dict[person_to_assign] > 0:
                    finalized_df.iloc[row, col] = person_to_assign
                    people_dict[person_to_assign] -= 1
                    remove_person_from_availability(person_to_assign, availability_df)

def fill_slots(finalized_df):
    """Main function to fill all slots in the finalized dataframe."""
    changed = True  # A flag to see if anything was changed in a full iteration
    while not all_slots_filled(finalized_df) and changed:
        changed = False  # Initially set to false for each iteration
        max_available_count = max(availability_df.applymap(len).values.ravel())
        for count in range(1, max_available_count+1):
            if fill_cells_by_count(count, finalized_df):  # Now it returns True/False based on changes
                changed = True
    fill_remaining_slots(finalized_df)



fill_slots(finalized_df)

finalized_df

,Monday,Tuesday,Wednesday,Thursday,Friday
10-11AM,NaN,NaN,NaN,NaN,NaN
11-12PM,NaN,NaN,NaN,NaN,NaN
12-1PM,NaN,NaN,NaN,NaN,NaN
1-2PM,NaN,NaN,NaN,NaN,NaN
2-3PM,NaN,NaN,NaN,NaN,NaN
3-4PM,NaN,NaN,NaN,NaN,NaN
4-5PM,NaN,NaN,NaN,NaN,NaN
5-6PM,NaN,NaN,NaN,NaN,NaN
6-7PM,NaN,NaN,NaN,NaN,NaN


In [3]:
'''
labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
columns = ["10-11AM", "11-12PM", "12-1PM", "1-2PM", "2-3PM", "3-4PM", "4-5PM", "5-6PM", "6-7PM"]
finalzied_df = pd.DataFrame(columns=labels, index=columns)

#Assign
def find_hours(input_str, my_dict):
    if input_str in my_dict:
        return my_dict[input_str]
    else:
        return None

def find_smallest_hours(keys_to_compare, data):
    # Set the initial minimum value and key
    min_val = float('inf')  # Set initial value to positive infinity so that any value in dict would be smaller
    min_key = None

    # Loop over the keys to compare
    for key_tuple in keys_to_compare:
        key = key_tuple[0]  # Extract the name from the tuple
        # Get the value for the current key
        if key in data:
            val = data[key]
            # If the current key's value is smaller, update min_val and min_key
            if val < min_val:
                min_val = val
                min_key = key
                
    if min_key is None:  # If no valid key was found
        return None, 0  # Return None for key and 0 for value

    return min_key, min_val


#def remove_person_from_df(string,person_name):
#    return ",".join([word for word in string.split(",") if word != person_name])

def delete_string_from_dataframe(df, target_string):
    for column in df.columns:
        df[column] = df[column].apply(lambda x: [word for word in x if word != target_string])
    return df

#print(finalzied_df)

smallest_cell_row=0
smallest_cell_col=0
smallest_cell_value=[]
for iterations in range(10):
    for smallest_value in range(10): # Can change it to the largest possible value of the matrix
        for col in range(1, 6):
            for row in range(0, 9):
                cell_values = availibility_df.iloc[row, col-1]  # treat cell values directly as a list
                if smallest_value == len(cell_values) and cell_values != []:
                    print(f"Available Guards for {row} row, {col} col: {cell_values}")
                    if find_smallest_hours(cell_values, people_dict)[1] > 0:
                        cell_temp = finalzied_df.iloc[row, col-1]
                        if pd.isna(cell_temp):
                            print(f"Assigning: {find_smallest_hours(cell_values, people_dict)[0]}")
                            assign_this_Person = find_smallest_hours(cell_values, people_dict)[0]
                            finalzied_df.iloc[row, col-1] = str(assign_this_Person)
                            people_dict[str(assign_this_Person)] -= 1
                            if find_smallest_hours(cell_values, people_dict)[1] == 0:
                                del people_dict[assign_this_Person]
                                availibility_df = delete_string_from_dataframe(availibility_df, assign_this_Person)
                            print(f"New Dict: {people_dict}")

#finalzied_df.insert(0, 'Busch Hours', ["10-11AM", "11-12PM", "12-1PM", "1-2PM", "2-3PM", "3-4PM", "4-5PM", "5-6PM", "6-7PM"])
finalzied_df.to_csv('Finalized Schedule.csv', index=False)
'''

Available Guards for 3 row, 1 col: [('Sriya', 'A1')]
Assigning: Sriya
New Dict: {'Alicia W': 3, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 3, 'Nikhil': 3, 'Parshva': 3, 'Shaan': 3, 'Sneha': 3, 'Sriya': 2, 'Tiffany': 3, 'Zeel ': 3}
Available Guards for 1 row, 2 col: [('Shaan', 'B')]
Assigning: Shaan
New Dict: {'Alicia W': 3, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 3, 'Nikhil': 3, 'Parshva': 3, 'Shaan': 2, 'Sneha': 3, 'Sriya': 2, 'Tiffany': 3, 'Zeel ': 3}
Available Guards for 3 row, 2 col: [('Parshva', 'B2')]
Assigning: Parshva
New Dict: {'Alicia W': 3, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 3, 'Nikhil': 3, 'Parshva': 2, 'Shaan': 2, 'Sneha': 3, 'Sriya': 2, 'Tiffany': 3, 'Zeel ': 3}
Available Guards for 0 row, 4 col: [('Nandini', 'B1')]
Assigning: Nandini
New Dict: {'Alicia W': 3, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 2, 'Nikhil': 3, 'Parshva': 2, 'Shaan': 2, 'Sneha': 3, 'Sriya': 2, 'Tiffany': 3, 'Zeel ': 3}
Available Guards for 0 row, 5 col: [('Zeel ',

In [13]:
availibility_df 
people_dict

{'Alicia W': 0,
 'Fatima': 1,
 'Gunjan': 0,
 'Luisa': 0,
 'Nandini': 0,
 'Nikhil': 0,
 'Parshva': 0,
 'Shaan': 0,
 'Sneha': 0,
 'Sriya': 0,
 'Tiffany': 0,
 'Zeel ': 0}

In [3]:
#print(people_dict)
#print(find_total_available_hours(availibility_df, people_dict))


labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
columns = ["10-11AM", "11-12PM", "12-1PM", "1-2PM", "2-3PM", "3-4PM", "4-5PM", "5-6PM", "6-7PM"]
finalzied_df = pd.DataFrame(columns=labels, index=columns)
#availibility_df.to_csv('Availibilities.csv',index=columns)


def is_cell_empty(row, column):
    cell_value = finalzied_df.iloc[row, column]
    return pd.isna(cell_value) or cell_value == '' or cell_value == []


for row in range(2, 9):
    for col in range(0, 5):
        while True:
            if is_cell_empty(row, col):
                if availibility_df.iloc[row][col]:  # Check if people are still left to be assigned or not.
                    temp_person = availibility_df.iloc[row][col][0][0]
                    if people_dict[temp_person] > 0:
                        finalzied_df.iloc[row][col] = temp_person
                        people_dict[temp_person] = people_dict[temp_person] - 1
                        print(f"Assigning {temp_person}")
                        print(people_dict)
                        break  # Exit the while loop and proceed to the next iteration of the row
                    elif people_dict[temp_person] == 0:
                        print(f"Not hours left for {temp_person}")
                        # Create a new list without the first tuple
                        updated_list = availibility_df.iloc[row][col][1:]
                        # Update the cell value with the new list using the .iat accessor
                        availibility_df.iat[row, col] = updated_list
                else:
                    break  # Exit the while loop if the list in the cell is empty
            else:
                break  # Exit the while loop if the cell is not empty


for row in range(0, 3):
    for col in range(0, 5):
        while True:
            if is_cell_empty(row, col):
                if availibility_df.iloc[row][col]:  # Check if the list is not empty
                    temp_person = availibility_df.iloc[row][col][0][0]
                    if people_dict[temp_person] > 0:
                        finalzied_df.iloc[row][col] = temp_person
                        people_dict[temp_person] = people_dict[temp_person] - 1
                        print(f"Assigning {temp_person}")
                        print(people_dict)
                        break  # Exit the while loop and proceed to the next iteration of the row
                    elif people_dict[temp_person] == 0:
                        print(f"Not hours left for {temp_person}")
                        # Create a new list without the first tuple
                        updated_list = availibility_df.iloc[row][col][1:]
                        # Update the cell value with the new list using the .iat accessor
                        availibility_df.iat[row, col] = updated_list
                else:
                    break  # Exit the while loop if the list in the cell is empty
            else:
                break  # Exit the while loop if the cell is not empty



#def assign_person():
    #resume coding from here, define this function
#availibility_df   
finalzied_df
#finalzied_df.to_csv('Finalized Schedule.csv',index=columns)
#Assigning hours to each cell
# def assign_hours(availibility_df):
#     for col in range(0, 5):
#         for row in range(0, 9):
#             #finalzied_df.iloc[row,col] = assign_person()
        


#assign_hours(availibility_df)



Step 1: True
Step 2: [('Tiffany', 'B1'), ('Sriya', 'A')]
Step 3: True
Assi 4: Tiffany
Assigning Tiffany
{'Alicia W': 3, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 3, 'Nikhil': 3, 'Parshva': 3, 'Shaan': 3, 'Sneha': 3, 'Sriya': 3, 'Tiffany': 2, 'Zeel ': 3}
Step 1: True
Step 2: [('Alicia W', 'B'), ('Nikhil', 'B')]
Step 3: True
Assi 4: Alicia W
Assigning Alicia W
{'Alicia W': 2, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 3, 'Nikhil': 3, 'Parshva': 3, 'Shaan': 3, 'Sneha': 3, 'Sriya': 3, 'Tiffany': 2, 'Zeel ': 3}
Step 1: True
Step 2: [('Sneha', 'B1'), ('Gunjan', 'A'), ('Nikhil', 'A')]
Step 3: True
Assi 4: Sneha
Assigning Sneha
{'Alicia W': 2, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 3, 'Nikhil': 3, 'Parshva': 3, 'Shaan': 3, 'Sneha': 2, 'Sriya': 3, 'Tiffany': 2, 'Zeel ': 3}
Step 1: True
Step 1: True
Step 2: [('Sriya', 'B1'), ('Zeel ', 'B2'), ('Nikhil', 'A')]
Step 3: True
Assi 4: Sriya
Assigning Sriya
{'Alicia W': 2, 'Fatima': 3, 'Gunjan': 3, 'Luisa': 3, 'Nandini': 3, 'Nikh

In [3]:
'''
{'Alicia W': 2,
 'Fatima': 3,
 'Gunjan': 0,
 'Luisa': 3,
 'Nandini': 0,
 'Nikhil': 3,
 'Parshva': 1,
 'Shaan': 2,
 'Sneha': 3,
 'Sriya': 0,
 'Tiffany': 3,
 'Zeel ': 1}
 
{'Alicia W': 0,
 'Fatima': 1,
 'Gunjan': 3,
 'Luisa': 2,
 'Nandini': 1,
 'Nikhil': 0,
 'Parshva': 0,
 'Shaan': 0,
 'Sneha': 0,
 'Sriya': 2,
 'Tiffany': 0,
 'Zeel ': 0}

 {'Alicia W': 0,
 'Fatima': 1,
 'Gunjan': 2,
 'Luisa': 0,
 'Nandini': 0,
 'Nikhil': 0,
 'Parshva': 0,
 'Shaan': 0,
 'Sneha': 0,
 'Sriya': 2,
 'Tiffany': 0,
 'Zeel ': 0}

 {'Alicia W': 0,
 'Fatima': 1,
 'Gunjan': 0,
 'Luisa': 0,
 'Nandini': 0,
 'Nikhil': 0,
 'Parshva': 0,
 'Shaan': 0,
 'Sneha': 0,
 'Sriya': 0,
 'Tiffany': 0,
 'Zeel ': 0}
 '''

finalzied_df

,Monday,Tuesday,Wednesday,Thursday,Friday
10-11AM,Zeel,Nandini,NaN,Nandini,Zeel
11-12PM,Shaan,Shaan,Shaan,NaN,NaN
12-1PM,Tiffany,Alicia W,Sneha,NaN,Sriya
1-2PM,Sriya,Parshva,Alicia W,NaN,Sriya
2-3PM,Alicia W,Sneha,Nikhil,Nandini,Gunjan
3-4PM,Luisa,Sneha,NaN,Gunjan,Luisa
4-5PM,Parshva,Nikhil,Parshva,Gunjan,Luisa
5-6PM,NaN,Nikhil,Tiffany,NaN,NaN
6-7PM,Zeel,Tiffany,Fatima,Fatima,NaN


In [5]:
#Scripts to Generate Random Schedules
'''
import pandas as pd
import random

# Define the strings to choose from
strings = ["YA", "YB", "YD", "B", "D", "N"]

# Define the number of rows and columns in the CSV file
num_rows = 9
num_cols = 5

# Create a list of randomly selected string values with 1's and 2's for each row and column
data = []
for i in range(num_rows):
    row = []
    for j in range(num_cols):
        s = random.choice(strings)
        if s != "N":
            num1 = 0
            num2 = 0
            if random.random() < 0.5:
                num1 = 1
            else:
                num2 = 1
            s = s + num1 * "1" + num2 * "2"
        row.append(s)
    data.append(row)

# Add 1 to three random strings and 2 to three other random strings
for i in range(3):
    row = random.randint(0, num_rows-1)
    col = random.randint(0, num_cols-1)
    if data[row][col] != "N":
        data[row][col] += "1"
for i in range(3):
    row = random.randint(0, num_rows-1)
    col = random.randint(0, num_cols-1)
    if data[row][col] != "N" and "1" not in data[row][col] and "2" not in data[row][col]:
        data[row][col] += "2"

fileName = 'person_10.csv'
# Create a Pandas DataFrame from the data and write it to a CSV file
df = pd.DataFrame(data)
df.to_csv(fileName, index=False)

# Print the contents of the CSV file to verify that it was saved correctly
with open(fileName, 'r') as f:
    print(f.read())


'''

'''
#Extracode from assigning logic
 #print(f"Available Guards for {row} row, {col} col: {cell_values} ")
                #print(f"Their hours: {find_smallest_hours(cell_values,people_dict)[1]}")
                print(f"New Dict: {people_dict}")
                print(f"Cell Values: {cell_values}")
                if find_smallest_hours(cell_values,people_dict)[1]>0:
                    #print(f"Assigning: {find_smallest_hours(cell_values,people_dict)[0]}")
                    assign_this_Person = find_smallest_hours(cell_values,people_dict)[0]
                    finalzied_df.iloc[row,col]= str(assign_this_Person)
                    people_dict[str(assign_this_Person)] -= 1
                    if find_smallest_hours(cell_values,people_dict)[1] == 0:
                        print(f"Zero hours left for: {assign_this_Person}")
                        del people_dict[assign_this_Person]
                        availibility_df = delete_string_from_dataframe(availibility_df,assign_this_Person)
                    print(f"New Dict: {people_dict}")
                else:
                    print("Error handeled")
'''                
people_dict
availibility_df

,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY
10-11pm,"[(Zeel , B1), (Luisa, B), (Nandini, B), (Sriya...","[(Tiffany, A2), (Nandini, B)]","[(Gunjan, B), (Luisa, B)]","[(Nandini, B1)]","[(Zeel , B1)]"
11-12pm,"[(Luisa, B1), (Shaan, B), (Sriya, A)]","[(Shaan, B)]","[(Luisa, B1), (Shaan, B), (Gunjan, A), (Nikhil...","[(Nandini, B), (Shaan, B)]","[(Nikhil, B1), (Zeel , B1), (Shaan, B)]"
12-1pm,"[(Tiffany, B1), (Sriya, A)]","[(Alicia W, B), (Nikhil, B)]","[(Sneha, B1), (Gunjan, A), (Nikhil, A)]",[],"[(Sriya, B1), (Zeel , B2), (Nikhil, A)]"
1-2pm,"[(Sriya, A1)]","[(Parshva, B2)]","[(Alicia W, B1), (Nandini, B1), (Parshva, B2),...",[],"[(Sriya, B1), (Nikhil, A)]"
2-3pm,"[(Alicia W, B1), (Sriya, A2), (Nikhil, B), (Gu...","[(Sneha, B1), (Nandini, B), (Sriya, B), (Gunja...","[(Nikhil, A), (Sriya, A)]","[(Nandini, B), (Gunjan, A), (Nikhil, A)]","[(Gunjan, A), (Nikhil, A)]"
3-4pm,"[(Alicia W, B1), (Luisa, B2), (Gunjan, A)]","[(Sneha, B2), (Nandini, B), (Gunjan, A), (Sriy...",[],"[(Gunjan, A), (Nikhil, A)]","[(Alicia W, B1), (Luisa, A1), (Gunjan, A), (Ni..."
4-5pm,"[(Parshva, B1), (Tiffany, B1), (Luisa, B), (Gu...","[(Nikhil, B1), (Parshva, B1), (Nandini, B), (S...","[(Parshva, B1), (Tiffany, A1), (Luisa, B)]","[(Parshva, B1), (Gunjan, A)]","[(Alicia W, B1), (Luisa, A1), (Sriya, B)]"
5-6pm,"[(Parshva, B2), (Luisa, B), (Gunjan, A)]","[(Nikhil, B1), (Shaan, B), (Sneha, B), (Zeel ,...","[(Tiffany, A1), (Sriya, B), (Gunjan, A)]","[(Parshva, B2), (Sriya, B)]","[(Alicia W, B1), (Sriya, B), (Luisa, A)]"
6-7pm,"[(Zeel , B2), (Luisa, B), (Sriya, B)]","[(Tiffany, B1), (Zeel , B1), (Nikhil, B), (Sri...","[(Fatima, A2), (Sriya, B), (Nikhil, A)]","[(Fatima, A2), (Sriya, B), (Luisa, A)]","[(Alicia W, B1), (Sneha, B2), (Sriya, B), (Tif..."


In [ ]:
'''
fileName = 'Finalzied Schedule.csv'
# Create a Pandas DataFrame from the data and write it to a CSV file
df = pd.DataFrame(data)
df.to_csv(fileName, index=False)
'''

"\nfileName = 'Finalzied Schedule.csv'\n# Create a Pandas DataFrame from the data and write it to a CSV file\ndf = pd.DataFrame(data)\ndf.to_csv(fileName, index=False)\n"